In [ ]:
!pip install langchain[all]
!pip install faiss-cpu
!pip install openai
!pip install huggingface_hub
!pip install transformers
!pip install gradio
!pip install unstructured
!pip install InstructorEmbedding
!pip install pypdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#1. Load the documents

In [ ]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('path', glob="*.txt")
loader.load()

In [ ]:
docs = loader.load()

In [ ]:
len(docs)

#2. Transform the documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 3500,
    chunk_overlap  = 500,
    length_function = len,
    is_separator_regex = False,
)

In [ ]:
texts =text_splitter.split_documents(docs)
print(texts[0])
print(texts[1])

In [ ]:
len(texts)

# 3. Generate the embeddings

In [ ]:
import openai
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = 'Your OpenAI-api key'

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

#4. Store it in vector store

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
!pip install tiktoken

In [ ]:
vectors= FAISS.from_documents(texts, embeddings )

In [ ]:
vectors

In [ ]:
#  The data is strore in the local folder. Use the path
vectors.save_local('path')

##Chains

● LLM

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
chat = ChatOpenAI(temperature=0)

● Prompts

In [ ]:
! pip install langchain openai chromadb langchainhub

In [ ]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

● Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key= 'answer')

● Retrievers

In [ ]:
retriever = vectors.as_retriever()

Now pass all the above LLm, retreval, Memory and prompt to the RAG chain Function


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain= (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | chat)


In [ ]:
chain= rag_chain.invoke("What is Task Decompositon")
chain

##Interface

In [ ]:
import gradio as gr
def chatllama(user_msg, history):
  result = rag_chain.invoke(user_msg )
  return result.content

# Add a title to the interface
demo = gr.ChatInterface(
    fn=chatllama,
    title="                          Chatbot                ",
)

demo.launch()
